In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

# Reading the Dataset

In [2]:
df = pd.read_csv("rawdata.csv")
print(df.shape)

(1146, 20)


In [3]:
df.head(5)

,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,Brawler2Name,Brawler2Tag,Brawler3Name,Brawler3Tag,Brawler4Name,Brawler4Tag,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,BEA,#222VJU0CU,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#2R9YVLQY,MAX,#22L8CLYQU,PENNY,#28U009JPY
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#8YVQRQP,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,JACKY,#2UPURLUG2,FRANK,#2UCV2R0LV,POCO,#22RURYRV0
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,PIPER,#PUG8CR80,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q


# Define Brawler Type Lists

In [4]:

sniper_brawlers = ['COLT','RICOCHET','8-BIT','PIPER','BROCK','BEA','PENNY','MR. P','SPIKE']
thrower_brawlers = ['DYNAMIKE','TICK','BARLEY']
heavyweight_brawlers = ['DARRYL','EL PRIMO','ROSA','BULL','FRANK','JACKY','BIBI']
fighter_brawlers = ['CARL','EMZ','BO','TARA','JESSIE','NITA','SHELLY']
support_brawlers = ['POCO','PAM','GENE','MAX','SPROUT','SANDY','GALE',]
assassin_brawlers = ['MORTIS','CROW','LEON']


# Add Brawler Types to the DataFrame
### Function to Assign Brawler Type

In [5]:
def assign_brawler_type(brawler):
    if brawler in sniper_brawlers:
        return 'SNIPER'
    elif brawler in thrower_brawlers:
        return 'THROWER'
    elif brawler in heavyweight_brawlers:
        return 'HEAVYWEIGHT'
    elif brawler in fighter_brawlers:
        return 'FIGHTER'
    elif brawler in support_brawlers:
        return 'SUPPORTER'
    elif brawler in assassin_brawlers:
        return 'ASSASSIN'
    else:
        return ''
    

### Add Brawler Types to DataFrame

In [6]:
df['Brawler1Type'] = df['Brawler1Name'].apply(assign_brawler_type)
df['Brawler2Type'] = df['Brawler2Name'].apply(assign_brawler_type)
df['Brawler3Type'] = df['Brawler3Name'].apply(assign_brawler_type)
df['Brawler4Type'] = df['Brawler4Name'].apply(assign_brawler_type)
df['Brawler5Type'] = df['Brawler5Name'].apply(assign_brawler_type)
df['Brawler6Type'] = df['Brawler6Name'].apply(assign_brawler_type)


# Check if same Type is used by multiple Players

In [7]:
counter = 0
for index, row in df.iterrows():
    if row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] != row['Brawler3Type']:
        df.loc[index, 'Brawler2Type'] = row['Brawler2Type']+'_2'
    elif row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] == row['Brawler3Type']:
        df.loc[index,'Brawler2Type'] = row['Brawler2Type']+'_2'
        df.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_3'
    elif row['Brawler1Type'] == row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] != row['Brawler3Type']:
        df.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
    elif row['Brawler1Type'] != row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] == row['Brawler3Type']:
        df.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
df



,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,...,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag,Brawler1Type,Brawler2Type,Brawler3Type,Brawler4Type,Brawler5Type,Brawler6Type
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,FIGHTER
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,MAX,#22L8CLYQU,PENNY,#28U009JPY,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SUPPORTER,SNIPER
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SNIPER,SNIPER
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,FRANK,#2UCV2R0LV,POCO,#22RURYRV0,SNIPER,SNIPER_2,SUPPORTER,HEAVYWEIGHT,HEAVYWEIGHT,SUPPORTER
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,HEAVYWEIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,2,20200530T035746.000Z,#8GQURUGG,brawlBall,Backyard Bowl,1,#8GQURUGG,CARL,MORTIS,#29CYUU0Y,...,PENNY,#82JPG02,CARL,#8GQURUGG,ASSASSIN,SNIPER,SNIPER_2,SNIPER,SNIPER,FIGHTER
1142,2,20200530T034530.000Z,#8GQURUGG,brawlBall,Backyard Bowl,1,#2Q0J0YJRJ,PIPER,BIBI,#CG9UG92G,...,CARL,#8GQURUGG,BO,#82JPG02,HEAVYWEIGHT,SUPPORTER,FIGHTER,SNIPER,FIGHTER,FIGHTER
1143,1,20200530T034324.000Z,#8GQURUGG,brawlBall,Backyard Bowl,1,#82JPG02,BO,PIPER,#2Q0J0YJRJ,...,TARA,#80JCU8QL,SPROUT,#20P922PJ9,SNIPER,FIGHTER,FIGHTER_2,SNIPER,FIGHTER,SUPPORTER
1144,1,20200530T034118.000Z,#8GQURUGG,brawlBall,Backyard Bowl,1,#8GQURUGG,CARL,PIPER,#2Q0J0YJRJ,...,PAM,#2RLQU2YC2,FRANK,#LYQLPGY2,SNIPER,FIGHTER,FIGHTER_2,ASSASSIN,SUPPORTER,HEAVYWEIGHT


# Find Frequent Combinations of Brawler Types

In [16]:
combinations = []
counter = 0
for index, row in df.iterrows():
    team1 = []
    team2 = []
    team1.append(row["Brawler1Type"])
    team1.append(row["Brawler2Type"])
    team1.append(row["Brawler3Type"])
    team2.append(row["Brawler4Type"])
    team2.append(row["Brawler5Type"])
    team2.append(row["Brawler6Type"])

    combinations.append(team1)
    combinations.append(team2)

In [17]:
te = TransactionEncoder()
te_ary = te.fit(combinations).transform(combinations)
dfFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
dfFreqSetsWinner

,,ASSASSIN,ASSASSIN_2,ASSASSIN_3,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,HEAVYWEIGHT_3,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
1,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
2288,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False
2289,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
2290,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False


In [18]:
frequent_itemsets_winner = apriori(dfFreqSetsWinner, min_support=0.01, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

In [19]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] >= 3) ].sort_values(by=["support"], ascending=False)




,support,itemsets,length
51,0.064572,"(SUPPORTER, FIGHTER, SNIPER)",3
47,0.057592,"(SUPPORTER, SNIPER, ASSASSIN)",3
53,0.053665,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
56,0.047120,"(SNIPER_2, SUPPORTER, SNIPER)",3
48,0.045375,"(HEAVYWEIGHT, FIGHTER, SNIPER)",3
55,0.040140,"(SNIPER_3, SNIPER_2, SNIPER)",3
49,0.039703,"(HEAVYWEIGHT, SUPPORTER, FIGHTER)",3
42,0.029232,"(FIGHTER, SNIPER, ASSASSIN)",3
57,0.028360,"(SUPPORTER_2, SUPPORTER, SNIPER)",3
46,0.026614,"(SNIPER_2, SNIPER, ASSASSIN)",3


# Find Frequent Winning Combinations of Brawler Types

In [20]:
winningCombinations = []
counter = 0
for index, row in df.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])
    elif row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])

    if len(team) != 0:
        winningCombinations.append(team)

In [21]:
te = TransactionEncoder()
te_ary = te.fit(winningCombinations).transform(winningCombinations)
dfFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
dfFreqSetsWinner

,,ASSASSIN,ASSASSIN_2,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
1,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
2,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
3,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
4,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
1109,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
1110,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False
1111,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False


In [22]:
frequent_itemsets_winner = apriori(dfFreqSetsWinner, min_support=0.01, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

In [23]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] >= 3) ].sort_values(by=["support"], ascending=False)


,support,itemsets,length
50,0.060198,"(SNIPER_3, SNIPER_2, SNIPER)",3
47,0.057502,"(FIGHTER, SUPPORTER, SNIPER)",3
48,0.049416,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
51,0.046721,"(SNIPER_2, SUPPORTER, SNIPER)",3
44,0.044924,"(ASSASSIN, SUPPORTER, SNIPER)",3
45,0.040431,"(HEAVYWEIGHT, FIGHTER, SNIPER)",3
46,0.035939,"(FIGHTER, HEAVYWEIGHT, SUPPORTER)",3
52,0.028751,"(SUPPORTER_2, SUPPORTER, SNIPER)",3
43,0.027853,"(ASSASSIN, SNIPER_2, SNIPER)",3
40,0.022462,"(ASSASSIN, FIGHTER, SNIPER)",3


# Find Frequent Loosing Combinations of Brawler Types

In [24]:
loosingCombinations = []
counter = 0
for index, row in df.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])
    elif row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])

    if len(team) != 0:
        loosingCombinations.append(team)

In [25]:
te = TransactionEncoder()
te_ary = te.fit(loosingCombinations).transform(loosingCombinations)
dfFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
dfFreqSetsWinner

,,ASSASSIN,ASSASSIN_2,ASSASSIN_3,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,HEAVYWEIGHT_3,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False
1109,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False
1110,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
1111,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False


In [26]:
frequent_itemsets_winner = apriori(dfFreqSetsWinner, min_support=0.01, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

In [27]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] >= 3) ].sort_values(by=["support"], ascending=False)

,support,itemsets,length
59,0.069182,"(SUPPORTER, FIGHTER, SNIPER)",3
55,0.069182,"(SUPPORTER, SNIPER, ASSASSIN)",3
61,0.058401,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
56,0.052111,"(HEAVYWEIGHT, FIGHTER, SNIPER)",3
64,0.048518,"(SNIPER_2, SUPPORTER, SNIPER)",3
57,0.045822,"(HEAVYWEIGHT, SUPPORTER, FIGHTER)",3
49,0.037736,"(FIGHTER, SNIPER, ASSASSIN)",3
50,0.035040,"(SUPPORTER, FIGHTER, ASSASSIN)",3
52,0.033243,"(HEAVYWEIGHT, SNIPER, ASSASSIN)",3
48,0.030548,"(HEAVYWEIGHT, FIGHTER, ASSASSIN)",3
